Vamos a intentar usar el código de la notebook `Priorizacion_con_red_chiquita` para la red completa de la notebook `05` o alguna subred interesante de ella.

In [1]:
%run imports.py
%run utils.py
%run leer_red.py


#########################################################
#
# OJO: Esto tiene un pico fuerte de uso de RAM! ~12 Gb
#
#########################################################

TENGO_MUCHA_RAM = True

#########################################################

/home/juan/miniconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%%time

red = leer_subred_nx() # Tarda ~20s
red_df = leer_subred_df()

print(f"{red.number_of_nodes():,} nodos en la red")
print(f"{red.number_of_edges():,} enlaces en la red")

37,823 nodos en la red
226,723 enlaces en la red
CPU times: user 19.2 s, sys: 308 ms, total: 19.5 s
Wall time: 19.5 s


In [3]:
import sys
from humanfriendly import format_size


def buscar_fenos_similares_por_LS(red, nodos_entrada):
    """Busca fenotipos similares en una red de networkx, entrando por los
    nodos_entrada. Devuelve un pandas.DataFrame de resultados."""
    
    adj_matrix = nx.adj_matrix(red).toarray()
    
    ram_a_usar = format_size(sys.getsizeof(adj_matrix))
    # print(f"WARNINIG: va a usar {ram_a_usar} de RAM")
    
    # En este paso la matriz original ocupa la RAM:
    adj_matrix_t = torch.FloatTensor(adj_matrix)

    labels = [0 if nodo in nodos_entrada else -1 for nodo in red.nodes()]
    labels_t = torch.LongTensor(labels)
    
    # Al instanciarse LabelSpreading, normaliza la matriz y
    # ahora hay dos matrices en memoria:
    label_spreading = LabelSpreading(adj_matrix_t, verbose=False)    
    label_spreading.fit(labels_t, alpha=.8)

    # Borramos las referencias a las dos matrices, este paso libera la RAM:
    del adj_matrix_t
    label_spreading.norm_adj_matrix = None
    
    resultado = pd.DataFrame({
        "nodos_entrada": "|".join(nodos_entrada),
        "nodo_salida": red.nodes(),
        "prediccion": label_spreading.flat_predictions,
    })
    
    return resultado.sort_values(by=["prediccion"], ascending=False)


nodos_de_categorias = [nodo for nodo in red.nodes()
                       if red.nodes[nodo]['tipo'] == 'categoria_fenotipo']
print(f"Borramos {len(nodos_de_categorias):,} nodos de categorias de fenotipo")
red.remove_nodes_from(nodos_de_categorias)

resultados = pd.DataFrame()
top_n = 40
fenos_entrada = red_df.fenotipo.value_counts().head(top_n).index

if TENGO_MUCHA_RAM:
    
    for feno in tqdm(fenos_entrada):
        resultado = buscar_fenos_similares_por_LS(red, nodos_entrada=[feno])
        resultados = pd.concat([resultados, resultado], axis=0, ignore_index=True)
        
resultados = resultados.reset_index(drop=True)
feno_a_categoria = red_df.set_index("fenotipo").categoria_fenotipo.to_dict()
resultados["tipo_de_nodo"] = resultados.nodo_salida.map(lambda nodo: red.nodes[nodo]['tipo'])
resultados["es_fenotipo"] = resultados.nodo_salida.isin(red_df.fenotipo)
resultados["categoria_fenotipo"] = resultados.nodo_salida.map(feno_a_categoria)
resultados["prediccion_log10"] = resultados.prediccion.map(np.log10)
resultados = resultados.query("nodos_entrada != nodo_salida")
resultados["k_nodo_entrada"] = resultados.nodos_entrada.map(red.degree)
resultados["k_nodo_salida"] = resultados.nodo_salida.map(red.degree)
resultados = resultados.sort_values(by=["nodos_entrada", "prediccion"], ascending=[True, False])
resultados = resultados.query("prediccion > 0")
resultados = resultados.reset_index(drop=True)
resultados["pred_rank"] = (
    resultados.groupby("nodos_entrada")
    .prediccion.rank(method="average", ascending=False)
)

fp = f"results/label_spreading.top{top_n}.tsv.gz"
resultados.to_csv(fp, index=False, sep="\t")

resultados

  0%|          | 0/40 [00:00<?, ?it/s]

Borramos 17 nodos de categorias de fenotipo


100%|██████████| 40/40 [05:20<00:00,  8.02s/it]
/home/juan/miniconda3/lib/python3.7/site-packages/pandas/core/base.py:1145: RuntimeWarning: divide by zero encountered in log10
  new_values = map_f(values, mapper)


,nodos_entrada,nodo_salida,prediccion,tipo_de_nodo,es_fenotipo,categoria_fenotipo,prediccion_log10,k_nodo_entrada,k_nodo_salida,pred_rank
0,Adventurousness,rs734073-T,2.473596e-02,alelo,False,NaN,-1.606671,167,2,6.5
1,Adventurousness,rs28646179-A,2.473596e-02,alelo,False,NaN,-1.606671,167,2,6.5
2,Adventurousness,rs72762188-A,2.473596e-02,alelo,False,NaN,-1.606671,167,2,6.5
3,Adventurousness,rs17201148-A,2.473596e-02,alelo,False,NaN,-1.606671,167,2,6.5
4,Adventurousness,rs60843830-C,2.473596e-02,alelo,False,NaN,-1.606671,167,2,6.5
...,...,...,...,...,...,...,...,...,...,...
1387435,Venous thromboembolism,rs112252907-T,2.651182e-08,alelo,False,NaN,-7.576560,179,1,34674.5
1387436,Venous thromboembolism,rs11086525-C,2.651182e-08,alelo,False,NaN,-7.576560,179,1,34674.5
1387437,Venous thromboembolism,rs62357000-T,2.651182e-08,alelo,False,NaN,-7.576560,179,1,34674.5
1387438,Venous thromboembolism,GSKIP,2.236638e-08,gen,False,NaN,-7.650404,179,1,34685.0


In [ ]:
# n_plots = len(fenos_entrada)
# n_cols = 2

# plot_width = 8
# plot_height = 10
# n_rows = (n_plots//n_cols)+1
# figsize = (plot_width * n_cols, plot_height * n_rows)
# top_n = 10

# for i, nodo_entrada in enumerate(fenos_entrada):
    
#     df = resultados.query("nodos_entrada == @nodo_entrada")
#     df = df.query("es_fenotipo").reset_index()
#     df["zero"] = 0
    
#     ax = plt.subplot(n_rows, n_cols, i+1)
    
#     df.head(top_n).plot.scatter(ax=ax, x="zero", y="prediccion_log10", color="CadetBlue", figsize=figsize, sharey=True)
#     ax.set_title(f"Nodos de entrada:\n{nodo_entrada[:40]}", fontsize=14)
#     ax.set_xticks([])
#     ax.set_xlabel(None)
#     ax.set_xlim((-1, 20))
    
#     for j, row in df.head(top_n).iterrows():
#         x, y = 1, row.prediccion_log10
#         ax.annotate(row.nodo_salida[:40], (x, y), verticalalignment="center", fontsize=14)
        
# plt.tight_layout()
# plt.savefig("results/test.png", dpi=110)